In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import json

def process_video_segments(data):
    new_data = {}

    for key, value in data.items():
        url = value["url"]
        start_time = value["start"]
        end_time = value["end"]
        annotations = value["annotations"]
        subset = value["subset"]

        # Calculate the total duration
        duration = math.ceil(end_time - start_time)

        for i in range(duration):
            clip_start = i
            clip_end = i+1

            clip_id = f"{key}_{i + 0:03d}"
            highlight = False
            label = None

            # Check if the 1-second clip falls into any segment
            for annotation in annotations:
                segment_start = math.floor(annotation["segment"][0])
                segment_end = math.ceil(annotation["segment"][1])

                if segment_start <= clip_start and clip_end <= segment_end:
                    highlight = True
                    label = annotation["label"]
                    break

            new_data[clip_id] = {
                "url": url,
                "subset": subset,
                "highlight": highlight,
                "label": label if label else "none"
            }

    return new_data

# # Example input JSON
# input_json = {
#     "QSGJHSYQI11P": {"url": "https://www.youtube.com/watch?v=Nvv1gdF_ed8", "start": 2222.0097889584763, "subset": "testing", "end": 2328.0097889584763,
#                      "annotations": [{"segment": [8.249999999999758, 9.64999999999973], "label": "strike"}, {"segment": [26.199999999999594, 28.999999999999535], "label": "ball"},
#                       {"segment": [45.19999999999989, 47.99999999999984], "label": "ball"}, {"segment": [65.19999999999933, 66.9499999999993], "label": "swing"},
#                        {"segment": [66.24999999999932, 67.64999999999928], "label": "strike"}, {"segment": [66.45999999999931, 67.64999999999928], "label": "hit"},
#                         {"segment": [66.9499999999993, 67.99999999999928], "label": "in play"}]}
# }



# # Process the input JSON
# output_json = process_video_segments(input_json)
# print(output_json)
# print(len(output_json))

In [ ]:
from tqdm import tqdm
def process_all_keys(data):
    """
    JSON 데이터의 모든 key에 대해 process_video_segments 함수를 실행하고, 진행 상황을 tqdm으로 시각화하는 함수.

    Args:
        data (dict): 처리할 JSON 데이터.

    Returns:
        dict: 모든 key에 대해 처리된 결과를 포함한 새로운 데이터.
    """
    all_results = {}

    for key in tqdm(data.keys(), desc="Processing keys"):
        segment_result = process_video_segments({key: data[key]})
        all_results.update(segment_result)

    return all_results

# JSON 파일 경로
json_file_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/mlb-youtube/data/mlb-youtube-continuous.json"

# JSON 파일 로드
with open(json_file_path, 'r') as f:
    json_data = json.load(f)

# 모든 key에 대해 process_video_segments를 실행
processed_results = process_all_keys(json_data)

# 처리된 결과를 JSON 파일로 저장
output_json_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second/seconds_result.json"
with open(output_json_path, 'w') as f:
    json.dump(processed_results, f)

print(f"모든 비디오 처리 완료! 결과가 {output_json_path}에 저장되었습니다.")

Processing keys: 100%|██████████| 2128/2128 [00:00<00:00, 3072.74it/s]


모든 비디오 처리 완료! 결과가 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second/seconds_result.json에 저장되었습니다.


In [ ]:
import json

# JSON 파일 경로
output_json_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second/seconds_result.json"

# JSON 파일 로드
with open(output_json_path, 'r') as f:
    json_data = json.load(f)

# training subset의 개수를 세는 함수
def count_training_subset(data):
    train = 0
    test =0
    for key, value in data.items():
        if value['subset'] == "training":
            train += 1
        else:
            test +=1
    return train, test

# training subset의 개수 계산
training_count = count_training_subset(json_data)

# 결과 출력
training_count

(187148, 47513)

In [ ]:
import numpy as np
import os

def split_npz_video(npz_path, output_dir,existing_files, file_prefix, fps=30):
    """
    npz 파일로 저장된 비디오 데이터를 1초 단위로 분할하여 저장하는 함수.

    Args:
        npz_path (str): 입력 npz 파일 경로.
        output_dir (str): 출력 npz 파일을 저장할 디렉토리 경로.
        file_prefix (str): 출력 파일명에 사용할 접두사.
        fps (int): 초당 프레임 수 (기본값은 30).
    """
    if file_prefix in existing_files:
        print(f"File {file_prefix} already exists, skipping...")

    else:
      # 출력 디렉토리가 존재하지 않으면 생성
      if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(output_dir)
        # npz 파일에서 비디오 데이터를 로드
        data = np.load(npz_path)
        video_frames = data['features']  # 비디오 프레임 데이터

        # 총 프레임 수와 비디오의 길이를 계산
        total_frames = video_frames.shape[0]
        duration = math.ceil(total_frames / fps)
        for i in range(duration):
            start_frame = i * fps
            end_frame = (i + 1) * fps

            # 1초짜리 비디오 클립 추출
            clip = video_frames[start_frame:end_frame]

            # 파일명을 지정하여 클립을 저장

            output_filename = f"{file_prefix}_{i + 0:03d}.npz"
            output_filepath = os.path.join(output_dir, output_filename)

            # npz 파일로 저장
            np.savez(output_filepath, features=clip)

        print(f"총 {duration}개의 1초짜리 클립이 저장되었습니다.")
      else:
        print(f"총 {output_dir}가 존재합니다")

In [ ]:
output_folder_1 = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second/"
files = [f for f in os.listdir(output_folder_1) if f.endswith('.npz')]
file_name_list = []
for file in files:
    file_name = file.split('_')[0]
    file_name_list.append(file_name)
existing_files = set(file_name_list)

In [ ]:
from tqdm import tqdm
import math
import json
def process_all_npz_files(input_folder, output_folder, existing_files):
    """
    입력 폴더 내의 모든 npz 파일에 대해 split_npz_video 함수를 실행하는 함수.

    Args:
        input_folder (str): npz 파일이 있는 입력 폴더 경로.
        output_folder (str): 분할된 npz 파일을 저장할 출력 폴더 경로.
    """

    npz_files = [f for f in os.listdir(input_folder) if f.endswith('.npz')]

    for npz_file in tqdm(sorted(npz_files), desc="Processing npz files"):
        npz_path = os.path.join(input_folder, npz_file)
        file_prefix = os.path.splitext(npz_file)[0]
        print(file_prefix)
        # 파일별 출력 폴더 설정
        output_dir = os.path.join(output_folder, file_prefix)
        # split_npz_video 실행
        split_npz_video(npz_path, output_dir,existing_files, file_prefix)

# 입력 폴더 및 출력 폴더 경로 설정
input_folder = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous/"
output_folder_1 = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second/"
output_folder = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/"

# 모든 npz 파일 처리
process_all_npz_files(input_folder, output_folder, existing_files)

Processing npz files:  10%|█         | 219/2092 [00:00<00:00, 2189.84it/s]

006FOJK7LI6A
File 006FOJK7LI6A already exists, skipping...
00IY1JT6SPK1
File 00IY1JT6SPK1 already exists, skipping...
00KJX29C9QLO
File 00KJX29C9QLO already exists, skipping...
018TV9B81MWH
File 018TV9B81MWH already exists, skipping...
02HX8DVSOKH6
File 02HX8DVSOKH6 already exists, skipping...
02WKL5ZOW5HW
File 02WKL5ZOW5HW already exists, skipping...
04WR4VWZ0SIX
File 04WR4VWZ0SIX already exists, skipping...
054JNXJQKDUO
File 054JNXJQKDUO already exists, skipping...
05L0G2D69KBN
File 05L0G2D69KBN already exists, skipping...
05PNZRODRDMW
File 05PNZRODRDMW already exists, skipping...
06A55DFT9V8J
File 06A55DFT9V8J already exists, skipping...
06CUGBTOYWX5
File 06CUGBTOYWX5 already exists, skipping...
07Z9KDOS7F3A
File 07Z9KDOS7F3A already exists, skipping...
08K8PIQTN1D3
File 08K8PIQTN1D3 already exists, skipping...
08X6PV2MZTA6
File 08X6PV2MZTA6 already exists, skipping...
0921BJL33L20
File 0921BJL33L20 already exists, skipping...
09DROCJMUM5G
File 09DROCJMUM5G already exists, skipping.

Processing npz files:  31%|███       | 646/2092 [00:00<00:00, 1903.22it/s]

총 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/7DBNF7K426W2가 존재합니다
7DX17D3KZDBA
총 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/7DX17D3KZDBA가 존재합니다
7EN5JBLQ5SB1
총 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/7EN5JBLQ5SB1가 존재합니다
7GAIHXZF2P8E
총 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/7GAIHXZF2P8E가 존재합니다
7GN1WGH4IBA5
총 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/7GN1WGH4IBA5가 존재합니다
7GNLTNW6RTMY
총 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/7GNLTNW6RTMY가 존재합니다
7IP28XX5V6VA
총 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/7IP28XX5V6VA가 존재합니다
7JB1HRTNO6RW
총 /content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/d

Processing npz files:  34%|███▎      | 702/2092 [01:02<03:44,  6.20it/s]  

...

총 125개의 1초짜리 클립이 저장되었습니다.
JMZDIX0NX73C
/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/JMZDIX0NX73C


Processing npz files:  52%|█████▏    | 1082/2092 [9:06:45<117:32:29, 418.96s/it]

총 115개의 1초짜리 클립이 저장되었습니다.
JN13HJK36GJ3
/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/JN13HJK36GJ3


Processing npz files:  52%|█████▏    | 1083/2092 [9:10:39<101:52:28, 363.48s/it]

총 122개의 1초짜리 클립이 저장되었습니다.
JN1D0NR3NTTN
/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous_second_2/JN1D0NR3NTTN


In [ ]:
from torch.utils.data import DataLoader

# QVHighlights 데이터셋 경로 설정

label_path = '/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/mlb-youtube/data/mlb-youtube-continuous.json'
video_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/video_features_continuous/"
audio_path = "/content/drive/MyDrive/Colab Notebooks/project/AIKU-summer/data/audio_features/"
# query_path = 'path/to/query'

# QVHighlights 데이터셋 인스턴스 생성
dataset = QVHighlights(
    label_path=label_path,
    video_path=video_path,
    audio_path=audio_path,
    # query_path=query_path
)

# DataLoader 생성
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

# 데이터 로딩 예제
for batch_idx, batch in enumerate(dataloader):
    videos = batch['video'].data  # 배치의 비디오 데이터
    audios = batch['audio'].data  # 배치의 오디오 데이터
    # queries = batch['query'].data  # 배치의 질의 데이터
    saliencies = batch['saliency'].data  # 배치의 살리언시 스코어
    metas = batch['meta'].data  # 배치의 메타 데이터 (예: qid, vid 등)

    # Optional: boundary가 있는 경우
    if 'boundary' in batch:
        boundaries = batch['boundary'].data

    # 여기서 모델에 입력하거나 추가적인 처리를 할 수 있습니다.
    print(f"Batch {batch_idx}:")
    print(f"  Video shape: {videos.shape}")
    print(f"  Audio shape: {audios.shape}")
    print(f"  Query shape: {queries.shape}")
    print(f"  Saliency shape: {saliencies.shape}")
    print(f"  Meta: {metas}")

    if 'boundary' in batch:
        print(f"  Boundary shape: {boundaries.shape}")

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-28-4b37080ee0d6>", line 20, in __getitem__
    video = self.get_video(idx)
  File "<ipython-input-28-4b37080ee0d6>", line 45, in get_video
    vid = self.label[idx]['vid']
KeyError: 59


In [ ]:
import torch
from torch.utils.data import Dataset
import os
import nncore
from nncore.parallel import DataContainer

class QVHighlights(Dataset):

    def __init__(self, label_path, video_path, audio_path):
        self.label = nncore.load(label_path)  # 라벨 데이터를 JSON 파일에서 로드

        self.label_path = label_path
        self.video_path = video_path
        self.audio_path = audio_path

    def __len__(self):
        return len(self.label)  # 데이터셋의 크기 반환

    def __getitem__(self, idx):
        video_segments, audio_segments, labels = [], [], []

        annotations = self.label[idx]['annotations']  # 비디오의 모든 주석(annotations)을 가져옵니다
        video = self.get_video(idx)
        audio = self.get_audio(idx)

        for annotation in annotations:
            start_time, end_time = annotation['segment']
            label = annotation['label']

            # 비디오와 오디오의 세그먼트 클립을 가져옵니다
            video_clip = self.get_segment(video, start_time, end_time)
            audio_clip = self.get_segment(audio, start_time, end_time)

            video_segments.append(video_clip)
            audio_segments.append(audio_clip)
            labels.append(label)

        video_segments = torch.stack(video_segments)
        audio_segments = torch.stack(audio_segments)

        # Saliency와 Boundary는 주석 데이터로부터 생성하지 않으므로 생략하거나 기본값을 설정합니다
        saliency = torch.ones(video_segments.size(0))
        boundary = None

        data = dict(
            video=DataContainer(video_segments),
            audio=DataContainer(audio_segments),
            labels=DataContainer(torch.tensor(labels), cpu_only=True),
            saliency=DataContainer(saliency, pad_value=-1),
            meta=DataContainer(self.label[idx], cpu_only=True))

        if boundary is not None:
            data['boundary'] = DataContainer(boundary, pad_value=-1)

        return data

    def get_video(self, idx):
        vid = self.label[idx]['vid']
        video_file = nncore.join(self.video_path, f'{vid}.npz')
        video_data = nncore.load(video_file)['features']
        video = torch.from_numpy(video_data).float()
        return video

    def get_audio(self, idx):
        vid = self.label[idx]['vid']
        audio_file = nncore.join(self.audio_path, f'{vid}.npy')
        audio_data = nncore.load(audio_file)
        return torch.from_numpy(audio_data).float()

    def get_segment(self, data, start_time, end_time):
        fps = 30  # 비디오의 초당 프레임 수 (필요에 따라 조정 가능)
        start_frame = int(start_time * fps)
        end_frame = int(end_time * fps)
        return data[start_frame:end_frame]

    def get_saliency(self, idx):
        # Saliency는 존재하지 않으므로 기본값으로 설정합니다
        return None

    def get_boundary(self, idx):
        # Boundary는 존재하지 않으므로 None을 반환합니다
        return None

    def evaluate(self, blob, **kwargs):
        num_samples, collected = len(blob), []
        blob = nncore.to_dict_of_list(blob)

        for i in range(num_samples):
            pred = dict(
                qid=blob['meta'][i][0]['qid'], vid=blob['meta'][i][0]['vid'])

            if 'saliency' in blob:
                pred['pred_saliency_scores'] = blob['saliency'][i][0].tolist()

            if 'boundary' in blob:
                pred['pred_relevant_windows'] = blob['boundary'][i][0].tolist()

            collected.append(pred)

        label = nncore.load(self.label_path)
        results = eval_qvhighlights(collected, label)['brief']

        return results
